In [1]:
import tasks.init_paths
import argparse
import time
import cv2

import os
import yaml
import pprint
import torch
import sys
from tqdm import tqdm
from six.moves import cPickle as pickle
from torchvision.ops.boxes import box_iou
import numpy as np

from models import *
from datasets.pascal_voc import VOCDetection

import datasets.voc_dataset_evaluator as voc_dataset_evaluator

from tasks.config    import cfg, load_config

import utils.data_transforms as dt_trans

from utils.data_samplers           import VisualizeSampler, collate_minibatch_visualize
from utils.detectron_weight_helper import load_detectron_weight
from utils.training_stats import TrainingStats

from utils.test_utils import box_results_for_corloc, box_results_with_nms_and_limit

import logging
from utils.logging import setup_logging
logger = logging.getLogger(__name__)

from pdb import set_trace as pause

In [30]:
'''
parser = argparse.ArgumentParser(description='Visualize detections')
parser.add_argument(
    '--dataset', default='voc2007trainval',
    help='training dataset')
parser.add_argument(
    '--cfg', default='configs/baselines/vgg16_voc2007.yaml', 
    dest='cfg_file', required=True,
    help='optional config file')
parser.add_argument(
    '--detections', default='outputs/detections/',
    help='the path for result file.')
parser.add_argument(
    '--output_dir', default='outputs/tests/',
    help='output directory to save the testing results.')
parser.add_argument(
        '--set', dest='set_cfgs',
        help='set config keys, will overwrite config in the cfg_file.'
            ' See lib/core/config.py for all options',
        default=[], nargs='*')

args = parser.parse_args([])
'''

"\nparser = argparse.ArgumentParser(description='Visualize detections')\nparser.add_argument(\n    '--dataset', default='voc2007trainval',\n    help='training dataset')\nparser.add_argument(\n    '--cfg', default='configs/baselines/vgg16_voc2007.yaml', \n    dest='cfg_file', required=True,\n    help='optional config file')\nparser.add_argument(\n    '--detections', default='outputs/detections/',\n    help='the path for result file.')\nparser.add_argument(\n    '--output_dir', default='outputs/tests/',\n    help='output directory to save the testing results.')\nparser.add_argument(\n        '--set', dest='set_cfgs',\n        help='set config keys, will overwrite config in the cfg_file.'\n            ' See lib/core/config.py for all options',\n        default=[], nargs='*')\n\nargs = parser.parse_args([])\n"

In [2]:
import easydict
 
args = easydict.EasyDict({
        "dataset": 'voc2007trainval',
        "cfg_file": '../configs/baselines/vgg16_voc2007.yaml',
        "detections": 'outputs/detections/',
        "output_dir": "outputs/tests/" 
})

In [3]:
cfg

AttrDict([('ROOT_DIR', '/home/shhan/oicr'),
          ('DATA_DIR', '/home/shhan/oicr/datasets')])

In [4]:
import os
os.getcwd()

'/home/shhan/oicr/code'

In [5]:
load_config(args.cfg_file)
cfg

AttrDict([('ROOT_DIR', '/home/shhan/oicr'),
          ('DATA_DIR', '/home/shhan/oicr/datasets'),
          ('TRAIN',
           AttrDict([('DATASET', 'voc_2007_trainval'),
                     ('SCALES', [480, 576, 688, 864, 1200]),
                     ('ITERATION_SIZE', 4),
                     ('IMS_PER_BATCH', 2),
                     ('BATCH_SIZE_PER_IM', 4096),
                     ('USE_FLIPPED', True),
                     ('PROPOSAL_FILES',
                      'data/selective_search_data/voc_2007_trainval.pkl'),
                     ('CROWD_FILTER_THRESH', 0),
                     ('GT_MIN_AREA', -1),
                     ('FREEZE_CONV_BODY', False),
                     ('FG_THRESH', 0.5),
                     ('BG_THRESH', 0.1),
                     ('SNAPSHOT_ITERS', 20000)])),
          ('ADAPTATIVE_SUP',
           AttrDict([('TYPE', 'log'),
                     ('LB', 100),
                     ('DO_TRICK', True),
                     ('ADAPTATIVE_IGN', True)])),
     

In [8]:
transforms = dt_trans.Compose([
	dt_trans.Normalize([102.9801, 115.9465, 122.7717]),
	dt_trans.HorizontalFlip(),
	dt_trans.Resize(),
	dt_trans.ToTensor(),
		
	])

if cfg.TRAIN.DATASET == 'voc_2007_trainval':
	dataset =  VOCDetection(cfg.DATA_DIR+'/voc/', year='2007', image_set='trainval', transforms=transforms) 
	
elif cfg.TRAIN.DATASET == 'voc_2012_trainval':
    dataset =  VOCDetection(cfg.DATA_DIR+'/voc/', year='2012', image_set='trainval', transforms=transforms)

/home/shhan/oicr/datasets/voc/VOCdevkit/VOC2007


In [11]:
dataset.proposals

[array([[  0.,   0.,  63.,   6.],
        [  0.,   0.,  86.,   7.],
        [  0.,   0.,  88.,   7.],
        ...,
        [349., 363., 370., 374.],
        [349., 363., 371., 374.],
        [349., 363., 377., 374.]]),
 array([[  0.,   0.,  40.,  37.],
        [  0.,   0.,  86.,  37.],
        [  0.,   0.,  43.,  38.],
        ...,
        [227., 322., 246., 332.],
        [ 61., 323.,  81., 332.],
        [356., 325., 375., 332.]]),
 array([[  0.,   0.,  23.,   8.],
        [  0.,   0.,  60.,  83.],
        [  0.,   0.,  70.,  83.],
        ...,
        [250., 364., 274., 374.],
        [ 43., 366.,  61., 374.],
        [ 84., 367., 120., 374.]]),
 array([[  0.,   0.,  30.,  20.],
        [  0.,   0.,  35.,  20.],
        [  0.,   0.,  47.,  20.],
        ...,
        [304., 320., 318., 332.],
        [150., 321., 187., 332.],
        [ 61., 322.,  81., 332.]]),
 array([[  0.,   0., 333.,  28.],
        [  0.,   0., 333.,  41.],
        [  0.,   0., 333.,  51.],
        ...,
        [

In [18]:
dataset.image_set

'trainval'

In [13]:
len(dataset.proposals)

5011

In [ ]:
def get_detections(file_name):
	file_name = os.path.abspath(file_name)
	with open(file_name, 'rb') as f:
		return pickle.load(f)['all_boxes']

In [ ]:
num_classes = cfg.MODEL.NUM_CLASSES
	
detections = get_detections(args.detections)

batchSampler = VisualizeSampler(len(dataset))


dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_sampler = batchSampler,
    num_workers   = cfg.DATA_LOADER.NUM_THREADS,
    collate_fn    = collate_minibatch_visualize)

In [ ]:
from pycocotools import mask as coco_mask

import datasets.transforms as T


class CocoDetection(torchvision.datasets.CocoDetection):
    def __init__(self, img_folder, ann_file, transforms, return_masks):
        super(CocoDetection, self).__init__(img_folder, ann_file)
        self._transforms = transforms
        self.prepare = ConvertCocoPolysToMask(return_masks)

    def __getitem__(self, idx):
        img, target = super(CocoDetection, self).__getitem__(idx)
        image_id = self.ids[idx]
        target = {'image_id': image_id, 'annotations': target}
        img, target = self.prepare(img, target)
        if self._transforms is not None:
            img, target = self._transforms(img, target)
        return img, target